<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>
<h2 align=center><font size = 4>Week 3 Peer-graded Assignment</font></h2>


In this notebook, we are going to explore and cluster the neighborhoods in Toronto.

### Step1: Scraping the Wikipedia 

We start by downloading & installing the BeautifulSoup library. We then use it to scrape the table in the wikipedia page and eventually port the neighborhoods of Toronto into a pandas dataframe.

In [1]:
#!pip install beautifulsoup4 #<-- Uncomment if you need to download & install beuatiful Soup
#import all the required modules for scraping
import requests
import bs4
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Modules successfully loaded')

Modules successfully loaded


In [2]:
# download the wikipedia html, raise error if unsuccessful
wikiPage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wikiPage.raise_for_status()

In [3]:
# Extract the table 
wikiSoup = bs4.BeautifulSoup(wikiPage.text)  #create soup from html text
zipTable = wikiSoup.select('table')[0]       #get the first table as a 'tag' element object
df1 = pd.read_html(str(zipTable))[0]          #get the first dataframe (theres only 1)
df1.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Replace all not assigned with NaN and drop not assigned Boroughs, then reset index
df1.replace("Not assigned", np.nan, inplace = True)
df1.dropna(subset=["Borough"], axis=0, inplace=True)
df1.reset_index(drop=True, inplace=True)
df1.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Check for any remaining Nan in Neighbourhood column
available_data = df1.notnull()
print(available_data["Neighbourhood"].value_counts())

True    103
Name: Neighbourhood, dtype: int64


No empty 'Neighbourhood' rows were found, so no further cleaning is required.

In [6]:
print("The shape of the final cleaned dataframe is: " , df1.shape)

The shape of the final cleaned dataframe is:  (103, 3)


### Step 2: Get the geographical coordinates of the neighborhoods using Geocoder, Geopy, etc.

Attempt 1 : Using Geocoder / Nominatim --> DID NOT WORK

In [7]:
#!pip install geocoder #<-- Uncomment if you need to download & install beuatiful Soup
import geocoder
g = geocoder.google('M1B Toronto')
coords = g.latlng
print(coords)
print(g)
#help(g)


None
<[REQUEST_DENIED] Google - Geocode [empty]>


In [8]:
coords = None
while(coords is None):
    g = geocoder.google('Toronto, Ontario')
    coords = g.latlng
    print(coords)

None
None
None
None


KeyboardInterrupt: 

Attempt 2 : Using Geopy --> DID NOT WORK on zipcodes. Gave random locations in Toronto

In [9]:
from geopy.geocoders import Nominatim 
address = "M1B, Toronto, Ontario"
geolocator = Nominatim(user_agent="Toronto_Explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of M1B, Toronto, Ontario are 43.6534817, -79.3839347.


In [10]:
print(location)

Toronto, Golden Horseshoe, Ontario, Canada


##### Solution: Read from provided csv:

In [11]:
# Use panadas read_csv function to read coordinates data straight into a new dataframe df2
fileUrl = "https://cocl.us/Geospatial_data"
df2 = pd.read_csv(fileUrl)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
print("The shape of the final cleaned dataframe is: " , df2.shape)

The shape of the final cleaned dataframe is:  (103, 3)


In [13]:
# Merge the dataframes on Postal Code
dff = pd.merge(df1,df2)

In [14]:
dff.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Check the size of our dataframe

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dff['Borough'].unique()),
        dff.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Step 3: Map the Neighbourhoods and check on the map of Toronto

Lets start by mapping all the neighbourhoods of Toronoto to verify all coordinate data is alright

In [16]:
#!pip install  folium # uncomment this line if you don't have folium installed

#Import the required modules for dealing with JSONs
import json
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Modules successfully loaded')

Modules successfully loaded


Now lets get the latitude and longitude values of using geopy

In [17]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Ontario_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


and now lets map the neighbourhoods on the city using folium

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dff['Latitude'], dff['Longitude'], dff['Borough'], dff['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
print('sweet!')

sweet!


### Step 4: Filter boroughs with 'Toronto' in its name

We choose to slice the original dataframe for only the boroughs with the word 'Toronto' in their name. So let's create a new dataframe as follows:

In [20]:
df_torr = dff[dff['Borough'].str.contains('Toronto')]
df_torr.reset_index(inplace = True)
df_torr.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [21]:
df_torr.shape

(39, 6)

Using the same coordinates for toronto we used before, lets map our filtered neighbourhoods and see if everything checks out.

In [22]:
# create map of Central Toronto using the same latitude and longitude values we got before
map_torr = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_torr['Latitude'], df_torr['Longitude'], df_torr['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torr)  
    
map_torr

In [23]:
print('double sweet!')

double sweet!


### Step 5: Use Foursquare to gather venues data for the neighbourhoods


Let's start by initializing our Foursquare API. (please copy your own credentials, i deleted mine after i ran the notebook)

In [24]:
CLIENT_ID = 'NQVSVFGS15XD2FKZVMPZDE33D2HSAANTARYBKOJZWUKOYIN3' # your Foursquare ID
CLIENT_SECRET = 'P255ITBVW1RFAGFEV1M3SQMOSFL14H1B1JUQWI5VTDC4SEE1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: NQVSVFGS15XD2FKZVMPZDE33D2HSAANTARYBKOJZWUKOYIN3
CLIENT_SECRET:P255ITBVW1RFAGFEV1M3SQMOSFL14H1B1JUQWI5VTDC4SEE1


Now we'll be defining a **getNearbyVenues** as follows. This function gets the top 100 venues for each neighbourhood in a json and cleans it to new dataframes

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's run the function to get a dataframe of all of the venues in the selected Toronto Neighbourhoods

In [29]:
toronto_venues = getNearbyVenues(names=df_torr['Neighbourhood'],
                                   latitudes=df_torr['Latitude'],
                                   longitudes=df_torr['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Let's check the size of the resulting dataframe which include all venues.

In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(1624, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for a random sample of neighbourhoods

In [31]:
toronto_venues.groupby('Neighbourhood').count().sample(5)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",14,14,14,14,14,14
Studio District,37,37,37,37,37,37
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
Lawrence Park,3,3,3,3,3,3
Church and Wellesley,75,75,75,75,75,75


and how many unique categories for all pulled venues

In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


<a id='item3'></a>


### Step 6: Analyze the foursquare data in preparation for clustering

Perform some wrangling to get venue counts by category

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#group rows by neighborhood  by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

and then print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.04
2        Beer Bar  0.04
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3       Nightclub  0.09
4       Pet Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Auto Workshop  0.06
2           Pizza Place  0.06
3            Comic Shop  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2  Rental Car Location  0.06
3        Boat or Ferry  0.06
4   Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coff

Using the **return_most_common_venue** function, we'll port the top 10 venues of each neighbourhood into a new dataframe


In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Basketball Stadium,Sporting Goods Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Grocery Store,Intersection,Bar,Bakery,Italian Restaurant,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Airport Terminal,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant


<a id='item4'></a>


### Step 7: Cluster the neighbourhoods using K-means, map them and then analyze them


Run _k_-means to cluster the neighborhood into 5 clusters.


In [37]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [38]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_torr

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Event Space,Restaurant
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant,Chinese Restaurant,Park
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,American Restaurant,Beer Bar,Gym,Moroccan Restaurant,Department Store
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


visualize the resulting clusters


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


Examine the 4 clusters one at a time


#### Cluster 1


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,-79.293031,0,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


#### Cluster 2


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Event Space,Restaurant
1,M7A,-79.389494,1,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant,Chinese Restaurant,Park
2,M5B,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,M5C,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,American Restaurant,Beer Bar,Gym,Moroccan Restaurant,Department Store
5,M5E,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Restaurant,Cheese Shop,Basketball Stadium,Sporting Goods Shop
6,M5G,-79.387383,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant
7,M6G,-79.422564,1,Grocery Store,Café,Park,Coffee Shop,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Baby Store,Nightclub
8,M5H,-79.384568,1,Coffee Shop,Café,Hotel,Restaurant,Gym,Bar,Thai Restaurant,Clothing Store,Concert Hall,Salad Place
9,M6H,-79.442259,1,Bakery,Pharmacy,Grocery Store,Park,Brewery,Middle Eastern Restaurant,Bank,Music Venue,Supermarket,Bar
10,M5J,-79.381752,1,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Sporting Goods Shop,Music Venue


#### Cluster 3


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,-79.416936,2,Music Venue,Garden,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5P,-79.411307,3,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
29,M4T,-79.383160,3,Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,M4W,-79.377529,3,Park,Playground,Trail,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
